In [5]:
import os
from dotenv import load_dotenv
from operator import itemgetter
load_dotenv()

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain.document_loaders.csv_loader import CSVLoader

from langchain.docstore.document import Document

from langchain_community.vectorstores import Neo4jVector

from langchain_openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda



In [8]:
embedding_model = OpenAIEmbeddings()
llm =  ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
url = "neo4j+ssc://ba68e959.databases.neo4j.io"
username = "neo4j"
password = os.getenv('NEO4J_PASSWORD') 

In [ ]:
vectorstore = Neo4jVector.from_existing_index(
        embedding_model,
        url=url,
        username=username,
        password=password,
        index_name="Chunk",
        )
retriever = vectorstore.as_retriever()

In [54]:
template = """Answer the question based only on the following context:
{context}
Question: {question}

Answer in the following language: {language}
When you have an Answer to response to say to the Human,create a link also, you MUST use the format:
```
<a href=\"" target=\"_blank\">our documentation</a>

```

"""
prompt = ChatPromptTemplate.from_template(template)


In [ ]:
chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
        
    }
    | prompt
    | llm
    
)
chain.invoke({"question": "Huongws", "language": "vietnamese"})


In [ ]:
retrieved_docs = retriever.invoke("Hướng dẫn")
retrieved_docs
